In [1]:
import os
from datetime import datetime
import time
import dotenv
import pandas as pd

import requests
import requests.auth

import praw

import openai

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# load secrets from .env into environment variables
dotenv.load_dotenv()

praw.__version__

'7.7.0'

See README.md
 - objective is to use OpenAI for named entity extraction to extract all the songs form [this reddit thread](https://www.reddit.com/r/AskReddit/comments/12viv4v/what_is_the_prettiest_song_you_ever_heard_in_your/) and make Spotify playlist
 - use Reddit PRAW API to download all the comments (get [Reddit API key](https://www.reddit.com/prefs/apps))
 - use OpenAI API with a prompt like, extract all the songs from this text to CSV get ([OpenAI API key](https://platform.openai.com/account/api-keys))
 - use Spotify API to make a playlist (get [Spotify API key](https://developer.spotify.com/documentation/web-api/tutorials/getting-started))
 - works, needed a lot of scrubbing, but about 1 day of work, wouldn't have been possible to do a 700-song playlist manually without a team of Mechanical Turks or something
 - If I wanted to go nuts, would process comments individually, save a file for each comment's extracted songs, would make it easier to track down what OpenAI gets wrong, have a resumable, retryable, repeatable process and 
 - Spotify playist is [here](https://open.spotify.com/playlist/08YFkbtTV6GBfNtjJ4PHDu?si=f4761d983ac84091) 
 
 needs a .env file per dot-env-template
 

In [2]:
# a thread 
submission = "12viv4v"

# minimum karma to process a reply 
minkarma = 5

# a prompt to apply to replies on the thread
prefix = """Define an example CSV file output as follows: 
"artist","song_title"
"The Beatles","Yesterday"
"Eagles","Hotel California"

Extract all song titles and artists from the following input, and return a CSV file output of the artists and song titles you extract from the input. If there were no songs extracted from the input, return "no songs found". the input is:
"""

# an output file to accumulate all the responses
savefile = 'bronze.txt'


## Get all comments from a reddit posting

In [3]:
def getPraw():
    return praw.Reddit(user_agent="prettiest_song/0.001", 
                       client_id=os.getenv('CLIENT_ID'), 
                       client_secret=os.getenv('CLIENT_SECRET'))


def getAll(r, submissionId, verbose=True):
    submission = r.submission(submissionId)
    submission.comments.replace_more(limit=None)
    commentsList=submission.comments.list()
    return commentsList


In [4]:
print(datetime.now())
r = getPraw()
res = getAll(r, submission)
print(datetime.now())

print("retrieved ", len(res), 'comments')

2023-04-29 21:09:54.154832
2023-04-29 21:53:54.813399
retrieved  24865 comments


In [5]:
# we have a list of comment objects
# filter comments with at least some karma
res3 = [r for r in res if r.score >= minkarma]
print('filtered to ', len(res3), 'comments')
res3[0].body, res3[0].score

filtered to  2501 comments


('Gymnopédies - Erik Satie', 6925)

## Extract artists and song titles using OpenAI

In [6]:
# check lengths of posts
for i in range(len(res3)):
    if len(res3[i].body) <3:
        print (i, res3[i].body)
    if len(res3[i].body) > 4096:
        print(i, len(res3[i].body))
        

472 4162
1534 W


In [7]:
# avg length
sum([len(r.body) for r in res3]) / len(res3)

106.75329868052779

In [8]:
print (res3[473].body[:500])

Top of the World


In [10]:
# for each comment object we will extract the body 
# then submit as part of a prompt to chatgpt
print(datetime.now())

openai.api_key = os.getenv('OPENAI_API_KEY')

slist = res3.copy()

maxchars = 5000  # max tokens is 4096 but we'll limit each prompt to 5000 chars
nposts = 100

# make sure no single post > maxchars + prefix
for i in range(len(slist)):
    if len(slist[i].body) > maxchars + len(prefix):
        print ("truncated ", i)
        slist[i].body = slist[i].body[:maxchars + len(prefix)]
        
outfile = open(savefile, 'w')
logdir = 'logs'
count = 0
c = 0


while(slist):  # still comments to process
    prompt = ""
    for _ in range(nposts):  # add up to 100 posts to the prompt
        if slist:
            if len(prompt) + len(slist[0].body) < maxchars:  
                body = slist.pop(0).body # in order, for better context 
#                 if len(body) <3:
#                     print (c, body)
                prompt += body
                prompt += " \n \n"
#                 c += 1            
            
    # retry loop, have received untrapped 502 error
    for _ in range(3):
        try:
            response = openai.ChatCompletion.create(
                model='gpt-3.5-turbo-0301',
                messages=[{"role":"user", 
                           "content": prefix + prompt}],
                temperature=0,
            )
        except Exception as error:
            print("An exception occurred:", error)
            print("looping...")
            time.sleep(5)
            continue  # try again
        break   # exception not triggered

    outfile.write(response['choices'][0]['message']['content'])
    outfile.write('\n\n')
    outfile.flush()
    
    with open("%s/%04d.log" % (logdir, count), 'w') as logfile:
        logfile.write(prompt)
        logfile.write('\n\n===========\n\n')
        logfile.write(response['choices'][0]['message']['content'])
 
    count += 1
#     print(c)
    print('.', end='')

outfile.close()
print()
print(datetime.now())



2023-04-29 22:13:41.301762
1534 W
..........................................................2023-04-29 22:24:54.367052


In [18]:
# may have to tweak the file to get it to load

df = pd.read_csv(savefile)
df


,artist,song_title
0,Don McLean,"Vincent (Starry, Starry Night)"
1,Lord Huron,The night we met
2,Simon & Garfunkel,Scarborough Fair
3,Neil Young,Harvest Moon
4,John Denver,Annie’s Song
...,...,...
1717,Goldfrapp,Felt Mountain
1718,Mono,Formica Blues
1719,The Beatles,Yesterday
1720,Eagles,Hotel California


In [19]:
df.drop_duplicates() \
    .dropna() \
    .sort_values(["artist", "song_title"]) \
    .to_csv('silver.csv', index=False)

print(len(df))
# tweak further to get to gold.csv



## Load into a Spotify playlist


In [14]:
client_credentials_manager = SpotifyClientCredentials(client_id=os.getenv('SPOTIFY_CLIENT_ID'), 
                                                      client_secret=os.getenv('SPOTIFY_CLIENT_SECRET'),
                                                      )

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


In [20]:
# check artists
df = pd.read_csv("silver.csv")

dedupe = {}
fail_list = []

for index, artist, title in df.itertuples():
    if artist in dedupe:
        continue
    dedupe[artist]=1
    query_str = 'artist:%s' % (artist)
    artist_results = sp.search(q=query_str, type='artist', limit=3, offset=0, market='US')
    artist_names = [artist['name'] for artist in artist_results['artists']['items']]
    if artist_names:
        if artist.lower() != artist_names[0].lower():
            print(artist, artist_names)
    else:
        fail_list.append((artist, title))
        print("not found:", artist, "-", title)


not found: .Hack//SIGN - Key of the Twilight
not found: 10,000 Days (Wings Pt. 2) - unknown
A-Ha ['Daryl Hall & John Oates', 'a-ha', 'half•alive']
not found: Acadian folk song - Evangeline
not found: Ageatus Byrjun - Sigur Rós
Al Stewart ['Alexander Stewart', 'Al Stewart', 'Alec Lee-Stewart']
Alan Parsons Project ['The Alan Parsons Project', 'The Alan Parsons Symphonic Project', 'The Alan Parsons Tribute Project']
Albinoni ['Tomaso Albinoni', 'Albinoni Tommaso', "Tomaso Albinoni's Orchestra Trieste"]
not found: Allison Kraus - When You Say Nothing at All
not found: Allison Krauss and Brad Paisley - Whiskey Lullaby
Amelia ['Amelia Moore', 'Amelia Grey', 'Amelia Magdalena']
America ['The All-American Rejects', 'America', 'American Authors']
not found: Andrew Lloyd Webber, Sarah Brightman, Paul Miles-Kingston - Pie Jesu
Babyface ['Babyface Ray', 'Babyface', 'BabyFaceWood']
Bach ['Johann Sebastian Bach', 'Bachman-Turner Overdrive', 'Tal Bachman']
Beach Boys ['The Beach Boys', 'Kalua Beach 

not found: The Crystal Ballroom - Ragged Wood
The Fugees ['The Re-Fugees']
The La’s ['The Kid LAROI', 'The Lacs', 'The Laurie Berkner Band']
The Mamas and the Papas ['The Mamas & The Papas', 'The Mamas & The Papas Experience', 'Karaoke - The Mamas & The Papas']
The OutKast ['The Outkasts', 'The Crazy Outkasts', 'KHALIDD THE OUTKAST']
The River ['The Last River Bottomed Fig Plucker', 'Them Riverbank Boys', 'The Riverside']
The Sunday's ['The Sundays', 'Sarah and the Sundays', 'The Sunday Shakes']
not found: The Trapeze Swinger - unknown
Tupac ['Techno Tupac', 'Tupac', 'Tupacshalore']
Unknown ['Unknown Mortal Orchestra', 'Unknown Brain', 'Unknown T']
not found: Untitled #3 (Samskeyti) - Sigur Rós
not found: Ursine Vulpine, Annaca - Wicked Game
not found: Vanilla Sky Soundtrack - Claire De Lune
Vaughan Williams ['Ralph Vaughan Williams', 'Vaughan Williams', 'Ralph Vaughan Vaughan Williams']
While my guitar gently weeps ['While My Guitar Gently Weeps with Strings']
Yaz ['Yazoo', 'Yazmin La

In [21]:
# check songs

df = pd.read_csv("silver.csv")
df.drop_duplicates() \
    .dropna() \
    .sort_values(["artist", "song_title"])

dedupe = {}
mylist = []
fail_list = []
artist_list, track_list, uri_list, album_list = [], [], [], []
orig_artist, orig_track = [], []

for index, artist, title in df.itertuples():
    query_str = 'artist:%s track:%s' % (artist, title)
    track_results = sp.search(q=query_str, type='track', limit=1, offset=0, market='US')
    results = track_results['tracks']['items']
    
    if results:
        r = results[0]
        # failsafe to never put same track twice
        if dedupe.get(r['id']):
            continue
        dedupe[r['id']]=True
        if title.lower() != r['name'].lower():
            print ("%s|%s : %s|%s" % (artist, title, r['artists'][0]['name'], r['name']))
        uri_list.append(r['uri'])
        artist_list.append(r['artists'][0]['name'])
        track_list.append(r['name'])
        album_list.append(r['album']['name'])
        orig_artist.append(artist)
        orig_track.append(title)
#         print('  ',
#               r['artists'][0]['name'],'|',
#               r['name'], '|',
#               r['album']['name'],'|',
#               r['album']['release_date'],'|',
#               r['popularity'])
    else:
        fail_list.append((artist, title))
        print("not found:", artist, "-", title)

not found: .Hack//SIGN - Key of the Twilight
not found: 10,000 Days (Wings Pt. 2) - unknown
A Perfect Circle|Three Libras : A Perfect Circle|3 Libras
not found: Acadian folk song - Evangeline
not found: Ageatus Byrjun - Sigur Rós
Albinoni|Adagio : Tomaso Albinoni|Adagio in G Minor (Arr. for Harp and Orchestra)
not found: Allison Kraus - When You Say Nothing at All
not found: Allison Krauss and Brad Paisley - Whiskey Lullaby
not found: Amelia - Joni Mitchell
Aphex Twin| "Avril 14th" : Aphex Twin|Avril 14th
not found: Aphex Twin -  "Avril 14th/QKThr"
not found: Aphex Twin -  "Lichen"
Aphex Twin| "Rhubarb" : Aphex Twin|Donkey Rhubarb
not found: Aphex Twin - Avrl 14
Arvo Pärt|Spiegel im Spiegel : Arvo Pärt|Spiegel im Spiegel - Version for Violin and Piano
Astrud Gilberto| "Photograph" : Astrud Gilberto|Photograph
not found: Aurora - Any song
Babyface|Every Time I Close My Eyes : Babyface|Every Time I Close My Eyes (with Kenny G)
Bach| "Air On G" : Johann Sebastian Bach|Orchestral Suite No.

not found: Hans Zimmer - Interstellar Intro
not found: Hans Zimmer and Benjamin Wallfisch -  "Wallace"
not found: Henri Mancini -  "Moon River"
Honeydrippers|Sea Of Love : The Honeydrippers|Sea of Love - 2006 Remaster
Hooverphonic|A New Stereophonic Sound Spectacular : Hooverphonic|Plus Profond
Howard Hanson|Symphony 2 : Howard Hanson|Symphony No. 2, Op. 30, "Romantic": I. Adagio - Allegro moderato
Hozier|Cherry Wine : Hozier|Cherry Wine - Live
Hozier|In A Week : Hozier|In a Week (feat. Karen Cowley)
not found: Humbling River - unknown
not found: I Stay Away - unknown
INXS| "Never Tear Us Apart" : INXS|Never Tear Us Apart
Ichiko Aoba| "Pilgrimage" : Ichiko Aoba|Pilgrimage
not found: Iron & Wine - Golden Slumbers
not found: Iron & Wine - This Coming Night
Iron & Wine|Trapeze Swinger : Iron & Wine|The Trapeze Swinger
Israel Kamakawiwo'ole|Somewhere Over The Rainbow : Israel Kamakawiwo'ole|Somewhere Over The Rainbow_What A Wonderful World
Israel Kamakawiwo'ole|White Sandy Beach : Israel K

Peter Gabriel|Red Rain : Peter Gabriel|Red Rain - 2012 Remaster
Peter Gabriel|Talk to Me : Peter Gabriel|Come Talk To Me
not found: Pharaoh Sanders - Love is Everywhere
Pink Floyd| "Great Gig In the Sky" : Pink Floyd|The Great Gig in the Sky
not found: Pirates of the Caribbean - Davy Jones Theme
not found: Portishead - Most songs
Queen| "Las Palabras De Amor" : Queen|Las Palabras De Amor (The Words Of Love) - Remastered 2011
Queen| "Love of my Life" : Queen|Love Of My Life - Remastered 2011
Queen|Good Old Fashioned Lover Boy : Queen|Good Old-Fashioned Lover Boy - Remastered 2011
Queen|Somebody to Love : Queen|Somebody To Love - Remastered 2011
Queen|Who Wants to Live Forever : Queen|Who Wants To Live Forever - Remastered 2011
Queensrÿche|Silent Lucidity : Queensrÿche|Silent Lucidity - Remastered 2003
not found: REM - Everybody Hurts
not found: REM - Night Swimming
Radiohead| "Daydreaming" : Radiohead|Daydreaming
Radiohead| "Nude" : Radiohead|Nude
Radiohead| "Reckoner" : Radiohead|Recko

The National|About Today : The National|About Today - 2021 Remaster
not found: The Noose - unknown
not found: The OutKast - You may die (intro)
not found: The Postal Service - Brothers On a Hotel Bed
not found: The Postal Service - Give Up
not found: The Postal Service - What Sarah Said
not found: The Postal Service - unknown
The River|unknown : Daniel & The River|Into The Great Unknown
The Smashing Pumpkins|Mellon Collie and the Infinite Sadness : The Smashing Pumpkins|Mellon Collie And The Infinite Sadness - Remastered 2012
The Smiths|Please Please Please Let Me Get What I Want : The Smiths|Please, Please, Please, Let Me Get What I Want - 2011 Remaster
not found: The Smiths - Stop me if you think that you’ve heard this one before
The Smiths|That Joke Isn’t Funny Anymore : The Smiths|That Joke Isn't Funny Anymore
The Smiths|There Is a Light That Never Goes Out : The Smiths|There Is a Light That Never Goes Out - 2011 Remaster
not found: The Trapeze Swinger - unknown
The Verve|Bitterswe

In [22]:
gold_df = pd.DataFrame({'input_artist': orig_artist,
                        'artist': artist_list,
                        'input_track': orig_track,
                        'track': track_list,
                        'album': album_list,
                        'uri': uri_list})
gold_df

,input_artist,artist,input_track,track,album,uri
0,311,311,Amber,Amber,Greatest Hits '93 - '03,spotify:track:51UtgWS4z1eMPuLQOzPtNH
1,A Perfect Circle,A Perfect Circle,Three Libras,3 Libras,Mer De Noms,spotify:track:5kHkaBN8OEQlmXfQkACxSt
2,A-Ha,a-ha,Take on Me,Take on Me,Hunting High and Low,spotify:track:2WfaOiMkCvy7F5fcp2zZ8L
3,ABBA,ABBA,Fernando,Fernando,Arrival,spotify:track:4BM8yJ0PzBi2ZewpMTOxtx
4,ABBA,ABBA,One of us,One Of Us,The Essential Collection,spotify:track:6zgtBUEkAfilJ2YEOvNexR
...,...,...,...,...,...,...
652,beabadoobee,beabadoobee,glue song,Glue Song,Glue Song,spotify:track:3iBgrkexCzVuPy4O9vx7Mf
653,grimes,Grimes,"""Symphonia IX""",Symphonia IX (My Wait Is U),Visions,spotify:track:76iqrqfj4k7Igq2WoKuTlS
654,unknown,Unknown92,Untitled #1,Untitled 1,GNO003,spotify:track:2ZrnqByl5UzrWOGyOOlAmf
655,unknown,Unknown Brain,unknown,Perfect 10 - Unknown Brain & RudeLies VIP,Perfect 10 (Unknown Brain & RudeLies VIP),spotify:track:7xddfUj4h1dwZ7ooFNJGMM


In [23]:
with pd.option_context("display.max_rows", 999):
    display(gold_df.loc[gold_df['input_artist'].str.lower() != gold_df['artist'].str.lower()])

,input_artist,artist,input_track,track,album,uri
10,Alan Parsons Project,The Alan Parsons Project,Time,Time,The Turn Of A Friendly Card (Expanded Edition),spotify:track:48yJZwYYDZX5GKFND7wDfC
12,Albinoni,Tomaso Albinoni,Adagio,Adagio in G Minor (Arr. for Harp and Orchestra),Notte Veneziana,spotify:track:47xdo9qYjAbOIVVBAqom1b
21,America,American Folk Channel,The Boxer,The Boxer,American Country Folk for the Trucks 2,spotify:track:6jLXjtanB0Yp82aAV7VGsN
39,Bach,Johann Sebastian Bach,"""Air On G""","Orchestral Suite No. 3 in D Major, BWV 1068: I...",Sleep for Baby,spotify:track:77GRzY2dy2IsSpSssSHPjq
43,Beach Boys,The Beach Boys,God Only Knows,God Only Knows - Mono,Pet Sounds (Original Mono & Stereo Mix),spotify:track:6iGU74CwXuT4XVepjc9Emf
48,Beatles,The Beatles,In My Life,In My Life - Remastered 2009,Rubber Soul (Remastered),spotify:track:3KfbEIOC7YIv90FIfNSZpo
50,Beethoven,Ludwig van Beethoven,7th Symphony 2nd Movement,"2nd Movement, 7th Symphony",Jerome Bixby's The Man From Earth: Original Mo...,spotify:track:7jrQurCEkjz7fkQ3Ed9JLh
51,Beethoven,Ludwig van Beethoven,Für Elise,"Bagatelle No. 25 in A Minor, WoO 59 ""Für Elise""","Für Elise, Bagatelle No. 25 in A Minor, WoO 59",spotify:track:3zLTPuucd3e6TxZnu2dlVS
54,Beyonce,Beyoncé,Plastic off the sofa,PLASTIC OFF THE SOFA,RENAISSANCE,spotify:track:6ufcuVInt0ocHrUimDjGlb
64,Bjork,Björk,"""Hyperballad""",Hyperballad,Post,spotify:track:4z1fNs2B7KndCsvyPgrhq5


In [ ]:
with pd.option_context("display.max_rows", 999):
    display(gold_df.loc[gold_df['input_track'].str.lower() != gold_df['track'].str.lower()])

In [ ]:
gold_df.iloc[29]

In [ ]:
# these are songs that look like covers or otherwise not the expected response from spotify search 
# (which is a bit wonky, doesn't like quotes and such)

bad_lookups = [
    32,422,556,564,575,621, 622,638,275
]

for i in bad_lookups:
    print(gold_df.iloc[i])
    
# add manually, plus 'not found'


In [ ]:
gold_df = gold_df.drop(
    axis='index',
    labels=bad_lookups)

gold_df[['artist', 'track']].to_csv('gold.csv', index=False)

with pd.option_context("display.max_rows", 999):
    display(gold_df)

In [25]:
# get playlist ids
# first create a playlist in UI to load songs
playlists = sp.user_playlists(os.getenv('SPOTIFY_USERNAME'))
while playlists:
    for i, playlist in enumerate(playlists['items']):
        if playlist['name'] != 'Reddit Prettiest Songs':
            continue
        print(playlist['id'])
        playlist_id = playlist['id']
        print("%4d %s %s" % (i + 1 + playlists['offset'], playlist['uri'],  playlist['name']))
    if playlists['next']:
        playlists = sp.next(playlists)
    else:
        playlists = None

08YFkbtTV6GBfNtjJ4PHDu
   1 spotify:playlist:08YFkbtTV6GBfNtjJ4PHDu Reddit Prettiest Songs


In [98]:
# must follow an oauth workflow to write a playlist in Spotify
# running this cell should request a spotify login and then redirect to an url
# paste whole url with id into form to authenticate

scope = "playlist-modify-public"

sp = spotipy.Spotify(auth_manager=spotipy.SpotifyOAuth(scope=scope,
                                                       client_id=os.getenv('SPOTIFY_CLIENT_ID'),
                                                       client_secret=os.getenv('SPOTIFY_CLIENT_SECRET'),
                                                       redirect_uri="https://druce.ai"
                                                      ))

In [ ]:
addlist = gold_df['uri'].to_list()
print (len(addlist))

while(addlist):
    sp.user_playlist_add_tracks(os.getenv('SPOTIFY_USERNAME'), 
                                playlist_id=playlist_id, 
                                tracks=addlist[-100:])
    addlist = addlist[:-100]
    print("added items, remaining ", len(addlist))


In [ ]:
# manually add the ones that weren't found for some reason


In [87]:
# can run again and add any new tracks, either because OpenAI is a bit random, or new replies in thread
results = sp.user_playlist(os.getenv('SPOTIFY_USERNAME'), playlist_id,
                                fields='tracks,next,name')
tracks = results['tracks']

playlist_dict_by_uri = {}
playlist_dict_by_str = {}

while True:
    for track_item in tracks['items']:
        track_dict = track_item['track']
        track_str = track_dict['artists'][0]['name']  + ' | ' + track_dict['name'][:15]
        uri = track_dict['uri']
        if track_str in playlist_dict_by_str:
            print(track_str)
        playlist_dict_by_str[track_str] = uri
        playlist_dict_by_uri[uri] = track_str
    # check if there are more pages
    if tracks['next']:
        tracks = sp.next(tracks)
    else:
        break

print (len(list(playlist_dict_by_str.keys())))
print (len(list(playlist_dict_by_uri.keys())))


676
676


In [91]:
gold_dict_by_uri = {}
gold_dict_by_str = {}
addlist = []
c = 0
for i, artist, track, uri in gold_df[['artist', 'track', 'uri']].itertuples():
    # print(artist, track, uri)
    track_str = artist + ' | ' + track[:15]
    if track_str not in playlist_dict_by_str:
        addlist.append([artist, track, uri])
        print(artist, track, uri)
    gold_dict_by_uri[uri]=track_str
    gold_dict_by_str['track_str']= uri
#     if track_str not in playlist_dict_by_str:
#         c += 1
#         print (c, track_str)
        
print(len(gold_dict_by_str.items()))
print(len(gold_dict_by_uri.items()))

a-ha Take on Me spotify:track:2WfaOiMkCvy7F5fcp2zZ8L
Adrián Berenguer Moonlight spotify:track:1vHan1eqIuiBG5jfliGO65
The Alan Parsons Project Time spotify:track:48yJZwYYDZX5GKFND7wDfC
Tomaso Albinoni Adagio in G Minor (Arr. for Harp and Orchestra) spotify:track:47xdo9qYjAbOIVVBAqom1b
alt-J Dissolve Me spotify:track:2Dv7PTwSoB17f3VFDIKw8m
Amen Dunes Song to the Siren spotify:track:5jHIijhpWocYfuM7lzNg72
American Folk Channel The Boxer spotify:track:6jLXjtanB0Yp82aAV7VGsN
Andrew Bird Beyond the Valley of the Three White Horses spotify:track:1DP0urxeM1qc2O9ky2vrDM
Andrew Bird Three White Horses spotify:track:6TAxmlqn13KElYfvNBmSPB
Animal Collective Banshee Beat spotify:track:6JOZHmPAfpB0drwuwFwPV9
Arvo Pärt Spiegel im Spiegel - Version for Violin and Piano spotify:track:4wudbbceOZKjRlhTtbHvAs
Astrud Gilberto Photograph spotify:track:1CTBSc0Ahu4tZUrxzyIluT
Ludwig van Beethoven 2nd Movement, 7th Symphony spotify:track:7jrQurCEkjz7fkQ3Ed9JLh
Big Thief Mary spotify:track:2l6aKj7mOu8MyYX4yFtlZ

In [92]:
addlist

[['a-ha', 'Take on Me', 'spotify:track:2WfaOiMkCvy7F5fcp2zZ8L'],
 ['Adrián Berenguer', 'Moonlight', 'spotify:track:1vHan1eqIuiBG5jfliGO65'],
 ['The Alan Parsons Project', 'Time', 'spotify:track:48yJZwYYDZX5GKFND7wDfC'],
 ['Tomaso Albinoni',
  'Adagio in G Minor (Arr. for Harp and Orchestra)',
  'spotify:track:47xdo9qYjAbOIVVBAqom1b'],
 ['alt-J', 'Dissolve Me', 'spotify:track:2Dv7PTwSoB17f3VFDIKw8m'],
 ['Amen Dunes', 'Song to the Siren', 'spotify:track:5jHIijhpWocYfuM7lzNg72'],
 ['American Folk Channel',
  'The Boxer',
  'spotify:track:6jLXjtanB0Yp82aAV7VGsN'],
 ['Andrew Bird',
  'Beyond the Valley of the Three White Horses',
  'spotify:track:1DP0urxeM1qc2O9ky2vrDM'],
 ['Andrew Bird', 'Three White Horses', 'spotify:track:6TAxmlqn13KElYfvNBmSPB'],
 ['Animal Collective', 'Banshee Beat', 'spotify:track:6JOZHmPAfpB0drwuwFwPV9'],
 ['Arvo Pärt',
  'Spiegel im Spiegel - Version for Violin and Piano',
  'spotify:track:4wudbbceOZKjRlhTtbHvAs'],
 ['Astrud Gilberto', 'Photograph', 'spotify:track:1

In [99]:
# clean up manually, covers, eagles of death metal etc
addlist = [
 ['Adrián Berenguer', 'Moonlight', 'spotify:track:1vHan1eqIuiBG5jfliGO65'],
 ['The Alan Parsons Project', 'Time', 'spotify:track:48yJZwYYDZX5GKFND7wDfC'],
 ['Tomaso Albinoni','Adagio in G Minor (Arr. for Harp and Orchestra)',
  'spotify:track:47xdo9qYjAbOIVVBAqom1b'],
 ['alt-J', 'Dissolve Me', 'spotify:track:2Dv7PTwSoB17f3VFDIKw8m'],
 ['Amen Dunes', 'Song to the Siren', 'spotify:track:5jHIijhpWocYfuM7lzNg72'],
 ['Andrew Bird', 'Three White Horses', 'spotify:track:6TAxmlqn13KElYfvNBmSPB'],
 ['Animal Collective', 'Banshee Beat', 'spotify:track:6JOZHmPAfpB0drwuwFwPV9'],
 ['Arvo Pärt',
  'Spiegel im Spiegel - Version for Violin and Piano',
  'spotify:track:4wudbbceOZKjRlhTtbHvAs'],
 ['Astrud Gilberto', 'Photograph', 'spotify:track:1CTBSc0Ahu4tZUrxzyIluT'],
 ['Ludwig van Beethoven',
  '2nd Movement, 7th Symphony',
  'spotify:track:7jrQurCEkjz7fkQ3Ed9JLh'],
 ['Big Thief', 'Mary', 'spotify:track:2l6aKj7mOu8MyYX4yFtlZ8'],
 ['Birdy', 'Skinny Love', 'spotify:track:4RL77hMWUq35NYnPLXBpih'],
 ['Black Sabbath',
  'Lonely Is the Word - 2008 Remaster',
  'spotify:track:2GSOOR5UXdDj9247nxyedO'],
 ['Blind Melon', 'No Rain', 'spotify:track:6txWz9UapYHVxEd7dDIHXT'],
 ['Blue Öyster Cult',
  'Fire of Unknown Origin',
  'spotify:track:3OXc6xDxasnl4uj2jl1q4Y'],
 ['Blur', 'Tender', 'spotify:track:42cxPm9jgbaxIVN77XA1m6'],
 ['Lil Bowser',
  'I Stole Peaches From Mario!',
  'spotify:track:601n7pnpUEebOo0SAl4lHt'],
 ['Brian Eno',
  'An Ending (Ascent) - Remastered 2005',
  'spotify:track:1vgSaC0BPlL6LEm4Xsx59J'],
 ['Frédéric Chopin',
  'Ballade No. 3 in A-Flat Major, Op. 47',
  'spotify:track:4hnxPFXpiHvPkdhdF4uNgf'],
 ['Chris Stapleton',
  'Tennessee Whiskey',
  'spotify:track:3fqwjXwUGN6vbzIwvyFMhx'],
 ['Clannad',
  "Theme from Harry's Game - Remastered",
  'spotify:track:2hcxPnu5RljXH0eMOtctuF'],
 ['Clay Aiken',
  'Bridge Over Troubled Water',
  'spotify:track:6pbQd4y7VwdLXQJWTBX2LZ'],
 ['Coldplay', 'Sparks', 'spotify:track:7D0RhFcb3CrfPuTJ0obrod'],
 ['Crosby, Stills, Nash & Young',
  'Our House',
  'spotify:track:2hitsKa8SthKhRJBXUHbIv'],
 ['Crosby, Stills, Nash & Young',
  'Helplessly Hoping',
  'spotify:track:40WeJU3odsbq1fXNbub4nh'],
 ['Crowded House',
  'Don’t Dream It’s Over - Home Demo',
  'spotify:track:0fiSpF9mvRFQWy0ca64d1g'],
 ['Cyndi Lauper', 'Time After Time', 'spotify:track:7o9uu2GDtVDr9nsR7ZRN73'],
 ['Daft Punk',
  'Touch (feat. Paul Williams)',
  'spotify:track:7oaEjLP2dTJLJsITbAxTOz'],
 ['Damien Rice',
  "The Blower's Daughter",
  'spotify:track:4B2lJinAkeNLSJjcq3dg8Q'],
 ['Damien Rice', 'Cannonball', 'spotify:track:3JiiqMYLbnHa1qys1JOpFK'],
 ['Damien Rice', 'Delicate', 'spotify:track:5mb6SzBnxv1ywFSH9V3uxd'],
 ['Dan Romer', 'End of the World', 'spotify:track:3YvGpUnRfRw3iJ5G4XPxGj'],
 ['Léo Delibes', 'Flower Duet', 'spotify:track:5K8jqeLAxZIqHR6e5w5so1'],
 ['Dido', 'It Comes and It Goes', 'spotify:track:3mutKSmOWCqFroNEFDl7ZA'],
 ['Don McLean', 'Castles in the Air', 'spotify:track:5RSeCwyPtZckl0GNNj7Rv5'],
 ['Dream Theater',
  'Stream of Consciousness',
  'spotify:track:4wIwfc4nbnH6Kxy0iIe2Up'],
 ['Eagles',
  'Hotel California - 2013 Remaster',
  'spotify:track:40riOy7x9W7GXjyGp4pjAv'],
 ['Eagles',
  'New Kid in Town - 2013 Remaster',
  'spotify:track:6Zyz8lsnMFpIrCTuvGurCB'],
 ['Elliott Smith', 'Between the Bars', 'spotify:track:1JbUMtVkQuaAPdHOvncVTm'],
 ['Elvis Presley', 'Suspicious Minds', 'spotify:track:1H5IfYyIIAlgDX8zguUzns'],
 ['Enya', 'Dreams Are More Precious', 'spotify:track:4p2Il3PinEajYSbbB4kt6i'],
 ['Eva Cassidy', 'Over the Rainbow', 'spotify:track:4Bh3FuYbXgUVyUH1Ht8zif'],
 ['Eva Cassidy', 'Time After Time', 'spotify:track:7ExSGiqt0W5xu3XoAwjnDm'],
 ['Evanescence',
  'Even In Death - 2016 Version',
  'spotify:track:4VO8CRItRuli3cnQLBDpvv'],
 ['Evanescence', 'Lacrymosa', 'spotify:track:1M8YN6ekSgCnjc5UckHYpq'],
 ['FKA twigs', 'Pendulum', 'spotify:track:4SMtnpWk1iFjunTtVpeR2y'],
 ['Fleet Foxes', 'Icicle Tusk', 'spotify:track:5gV0I7SE5136QcnSpxdlvw'],
 ['Fleetwood Mac',
  'Albatross - 2018 Master',
  'spotify:track:3iTkd7x5X2LIVKu7JNwqHR'],
 ['Florence + The Machine',
  'Breath Of Life',
  'spotify:track:2UhYsS3kTA9BPdVNIapma0'],
 ['Frank Sinatra',
  'Fly Me To The Moon (In Other Words)',
  'spotify:track:5b7OgznPJJr1vHNYGyvxau'],
 ['Franz Schubert',
  'Ave Maria, D. 839',
  'spotify:track:4JsMpgsep6XpuHSzKCTxCz'],
 ['Frente!',
  'Bizarre Love Triangle - 2014 Remaster',
  'spotify:track:7sgi66biRYpAXuRZJBDuli'],
 ['Giacomo Puccini',
  'Turandot, SC 91, Act III: Nessun dorma!',
  'spotify:track:74WjYdm3Lvbwnds4thYPUU'],
 ['Giacomo Puccini',
  'Puccini: Madama Butterfly, Act 2: "Un bel dì vedremo" (Butterfly)',
  'spotify:track:69EKwvg7nkdOO3RmeUMJZG'],
 ['The Goo Goo Dolls', 'Name', 'spotify:track:1G8jae4jD8mwkXdodqHsBM'],
 ['Gordon Lightfoot',
  'Circle of Steel',
  'spotify:track:4gOC5ylMlc3DGohuXIXt1s'],
 ['Guillemots', 'Little Bear', 'spotify:track:5ciM4wLYPh252DUNcitN2I'],
 ["Guns N' Roses", 'November Rain', 'spotify:track:3YRCqOhFifThpSRFJ1VWFM'],
 ['Hans Zimmer',
  'What Are You Going to Do When You Are Not Saving the World?',
  'spotify:track:6PHMAqyTi2zjs6aXneg0EC'],
 ['Harry Chapin', 'Taxi', 'spotify:track:7qnxmC6qdYkjcoCJCeD8Jx'],
 ['The Honeydrippers',
  'Sea of Love - 2006 Remaster',
  'spotify:track:0VMGij4wSGBM5pSTcqjxeD'],
 ['Hooverphonic', 'Plus Profond', 'spotify:track:4ZvbDx8HuPvAAOzYGcCNWm'],
 ['Howard Hanson',
  'Symphony No. 2, Op. 30, "Romantic": I. Adagio - Allegro moderato',
  'spotify:track:6wI079MMvFDriadIFk4KOr'],
 ['Hozier', 'Work Song', 'spotify:track:5TgEJ62DOzBpGxZ7WRsrqb'],
 ['Hudson Mohawke', 'Cbat', 'spotify:track:1JGGsCZ9zhgPJIsPy0pwt9'],
 ['Iron & Wine', 'Passing Afternoon', 'spotify:track:4mRLQVrkJqNjvBxMXSQ53X'],
 ['Iron & Wine',
  'The Trapeze Swinger',
  'spotify:track:0EAvKXjChn1Rc3QmfYXevy'],
  ["Israel Kamakawiwo'ole",
  "White Sandy Beach Of Hawai'i",
  'spotify:track:3PG9Q89FOydXO8rJesUewH'],
 ['Jaakko Aukusti',
  'What If All Else Fails?',
  'spotify:track:1Qp6g0Q18sF9dtrWs1gIJU'],
 ['Jay Ungar', 'Ashoken Farewell', 'spotify:track:2s6pqLeVialgt5l5TTSeas'],
 ['Jeff Buckley',
  'Everybody Here Wants You',
  'spotify:track:2bcvooA6HEmVUneEGJnNZD'],
 ['Jerry Garcia Band',
  'Reuben and Cherise',
  'spotify:track:1g5gLvo1we4q9YzCbGJJte'],
 ['Jimi Hendrix', 'Angel', 'spotify:track:0QenQiXnBs7s9fBWGluD17'],
 ['Johann Sebastian Bach',
  "Jesu Joy of Man's Desiring, BWV 147 (Arr. Hess)",
  'spotify:track:4tqUJF3JMttpt6BvVuJRLx'],
 ['John Craigie', 'I Am California', 'spotify:track:27ytYULTu6QSZBhGaOKq9i'],
 ['Johnny Cash',
  'If You Could Read My Mind',
  'spotify:track:6lZWRlcTC5XVogPScEaG9o'],
 ['José González', 'Teardrop', 'spotify:track:02C433ZVoi5cPFOSEKKzUX'],
 ['Judas Priest', 'Fire Burns Below', 'spotify:track:4sAOwd8m4YnGNoK4wxccUv'],
 ['Justin Hurwitz',
  'Mia & Sebastian’s Theme',
  'spotify:track:1Vk4yRsz0iBzDiZEoFMQyv'],
 ['Kyle Landry',
  "Mia and Sebastian's Theme",
  'spotify:track:6u4alUtWbsXjZQy9IfHd0r'],
 ['Ms. Lauryn Hill', 'Ex-Factor', 'spotify:track:2VjXGuPVVxyhMgER3Uz2Fe'],
 ['Led Zeppelin',
  'All My Love - Remaster',
  'spotify:track:5hhVpGIBlqAU5yJEOmrk5o'],
 ['Léo Delibes',
  'Delibes: Lakmé, Act 1: "Dôme épais, le jasmin" (Lakmé, Mallika)',
  'spotify:track:3KhSh5mImnRARr3GC2eaui'],
 ['Lesley Garrett',
  'Schönberg, Claude-Michel: I Dreamed a Dream (from "Les Misérables")',
  'spotify:track:02SO3ikjtfG1avKhU3jQdw'],
 ['Lisa Gerrard',
  'Sanvean - I Am Your Shadow',
  'spotify:track:6A1mKZwhIkYzL88GHX31YX'],
 ['Lord Huron',
  'The Ghost on the Shore',
  'spotify:track:0DSUmQ49MgUyV0vAG2yidb'],
 ['Mariah Carey', 'Always Be My Baby', 'spotify:track:2aBxt229cbLDOvtL7Xbb9x'],
 ['Massive Attack', 'Exchange', 'spotify:track:2HuMQkNVpFIsur2cRWWQmX'],
 ['Massive Attack', 'Black Milk', 'spotify:track:1Rezzt36ybaT2ZbDZpv83D'],
 ['Massive Attack', 'Group Four', 'spotify:track:0pWrU4My52DJ75GnZKDlv8'],
 ['Matisyahu', 'One Day', 'spotify:track:0nCZDQrlWA149QdGTdlsFU'],
 ['Maurice Ravel',
  'Ravel: Pavane pour une infante défunte, M. 19',
  'spotify:track:3MRQn2RYo2VLYMoStnLRxu'],
 ['Max Elto',
  'Citylights - Radio Edit',
  'spotify:track:6U819wVYsM6pr6LSQpFPAK'],
 ['Max Richter',
  'Recomposed By Max Richter: Vivaldi, The Four Seasons: Spring 1 - 2012',
  'spotify:track:4gmhjAsvL6c7se2PBh7dHw'],
 ['Mazzy Star',
  'Look On Down From The Bridge',
  'spotify:track:010ALbbh5KlQ4fKWgSdhRd'],
 ['Mercedes Sosa',
  'Alfonsina Y El Mar',
  'spotify:track:6Q3ozAXkxLpKQy6sc8L0TY'],
 ['Message To Bears',
  'You Are a Memory',
  'spotify:track:5cWSpmTfItDuppACMrE3I9'],
 ['Metric', 'Now or Never Now', 'spotify:track:6gSNEJjZm0DrGRRypBm24k'],
 ['múm',
  'Green Green Grass Of Tunnel',
  'spotify:track:4cwCN5oe8NtPb1orgeyMWM'],
 ['My Chemical Romance',
  'Welcome to the Black Parade',
  'spotify:track:5wQnmLuC1W7ATsArWACrgW'],
 ['Nasko', 'See You Again', 'spotify:track:2BsIe7BHWCdiK1azJyFzuC'],
 ['Nat King Cole', 'Smile', 'spotify:track:1oDCK7PW72XEZ1pE5rh87A'],
 ['Nat King Cole', 'Stardust', 'spotify:track:3npHR2nReF5bhCvvGiNwcL'],
 ['Nick Cave & The Bad Seeds',
  'Into My Arms - 2011 Remastered Version',
  'spotify:track:407ltk0BtcZI8kgu0HH4Yj'],
 ['Nils Frahm', 'Familiar', 'spotify:track:5Yy0R2CLFPXfrBPLnsI1zN'],
 ['Nine Inch Nails',
  'Just Like You Imagined',
  'spotify:track:184ImITRjcX2h1awZ4WVT1'],
 ['Nine Inch Nails', 'A Warm Place', 'spotify:track:1HsjoAytWxKDxjAV6dLGxW'],
 ['Nine Inch Nails', 'Hurt', 'spotify:track:1lo9k4PrxFd5Np4cAxXoKo'],
 ['ONR',
  'Must Stop (Falling in Love) [feat. Sarah Barthel of Phantogram]',
  'spotify:track:6qomQtETB8ovqxCJzwJS74'],
 ['Oliver Tank',
  'Last Night I Heard Everything in Slow Motion',
  'spotify:track:0QPhzxGzTx0oBioSGsQI6U'],
 ['Olivia Millerschin', 'Carry Me', 'spotify:track:0m9CKuKNadS4MGLyNv7Fx8'],
 ['Orbital', 'Halcyon and On and On', 'spotify:track:5xcunlfaZvD9BDQsLONI7A'],
 ['Orville Peck', 'Summertime', 'spotify:track:5bKEtGnNsKE8c7ew66rUld'],
 ['Otis Redding',
  "(Sittin' On) the Dock of the Bay",
  'spotify:track:3zBhihYUHBmGd2bcQIobrF'],
 ['Pantera', 'Floods', 'spotify:track:1L2ZkXbRX00ZiaUDuMMgf7'],
 ['Pantera', 'The Sleep', 'spotify:track:49e9eCJwK2aMkPk4rPX3fD'],
 ['Patrick Watson',
  'Je te laisserai des mots',
  'spotify:track:0V5cvmTKsYmF5FmGGEAfmS'],
 ['Patsy Cline',
  'Crazy - Single Version',
  'spotify:track:3zpj9dvJABiyMrmLCPw6i8'],
 ['Patsy Cline',
  'I Fall To Pieces - Single Version',
  'spotify:track:6zFsz86nAj5gcvzyVqNL3k'],
 ['Paul Simon', 'Kodachrome', 'spotify:track:3f0U5NaD1bCk8nmKpn2ZJY'],
 ['Pearl Jam', 'Black', 'spotify:track:5Xak5fmy089t0FYmh3VJiY'],
 ['Pearl Jam', 'Just Breathe', 'spotify:track:6i81qFkru6Kj1IEsB7KNp2'],
 ['Pearl Jam', 'Yellow Ledbetter', 'spotify:track:3bE5slaVEfaDreqARl6k4M'],
 ['Periphery', 'Have a Blast', 'spotify:track:1mcbjDMzRWVqI3YeiYdUrR'],
 ['Peter Gabriel', 'Blood Of Eden', 'spotify:track:4CO8g2Psbp1knUsiiF7Sa5'],
 ['Peter Gabriel', 'The Book Of Love', 'spotify:track:4TGjfiGfhS8OKXHdghtMh0'],
 ['Peter Gabriel', 'Family Snapshot', 'spotify:track:3u6POVv8GphO4WUXoZPtbK'],
 ['Peter Gabriel',
  'Red Rain - 2012 Remaster',
  'spotify:track:3RpKyeQWrZFwX2fomsq2Y8'],
 ['Peter Gabriel', 'Come Talk To Me', 'spotify:track:0oUMhkFaJR1T203cxz2r9K'],
 ['Phoebe Bridgers', 'Funeral', 'spotify:track:3rJDsZjIaCfJQSqRUvl0BM'],
 ['Phoebe Bridgers', 'Moon Song', 'spotify:track:46RNrAkGsqWTDrv2ZPOAbx'],
 ['Pink Floyd',
  'The Great Gig in the Sky',
  'spotify:track:2TjdnqlpwOjhijHCwHCP2d'],
 ['Pink Floyd', 'Fearless', 'spotify:track:7AalBKBoLDR4UmRYRJpdbj'],
 ['Pink Floyd', 'High Hopes', 'spotify:track:5a4MgIUSf9K8wXLSm6xPEx'],
 ['Pink Floyd', 'Stay', 'spotify:track:2loweLvzkHJTe7sbbw1nO8'],
 ['Pink Floyd', 'Wish You Were Here', 'spotify:track:6mFkJmJqdDVQ1REhVfGgd1'],
 ['Portishead', 'The Rip', 'spotify:track:5KX2DSPC6aCA0pdDidTmBC'],
 ['Pretty Lights', 'Yellow Bird', 'spotify:track:1r5J8bYOWq1Dal5jMQ06WX'],
 ['Prince', 'Purple Rain', 'spotify:track:54X78diSLoUDI3joC2bjMz'],
 ['Purity Ring', 'fineshrine', 'spotify:track:5aJmSzmuhLI6fr1Rm0EnkS'],
 ['Queen',
  'Las Palabras De Amor (The Words Of Love) - Remastered 2011',
  'spotify:track:5hHpNI203ne4TmVP3c9LPq'],
 ['Queen',
  'Love Of My Life - Remastered 2011',
  'spotify:track:790YJcgHlN3SaosQCHlWzn'],
 ['Queen',
  'Good Old-Fashioned Lover Boy - Remastered 2011',
  'spotify:track:1mnQiO568zXIrUncttTZGp'],
 ['Queen',
  'Somebody To Love - Remastered 2011',
  'spotify:track:6cFZ4PLC19taNlpl9pbGMf'],
 ['Queen',
  'Who Wants To Live Forever - Remastered 2011',
  'spotify:track:00QAndVDVfNqNWYdWAhEan'],
 ['Queensrÿche',
  'Silent Lucidity - Remastered 2003',
  'spotify:track:6OSyCAmXT4Gkd3OQ2aPOaF'],
 ['Racer X', 'Scarified', 'spotify:track:6vNdmNxQHacCmZy0k2nuPq'],
 ['Radical Face', 'Always Gold', 'spotify:track:1gOTLXloeEsw7cbARwnEvs'],
 ['Radiohead',
  'Weird Fishes/ Arpeggi',
  'spotify:track:4wajJ1o7jWIg62YqpkHC7S'],
 ['Rainbow', 'Rainbow Eyes', 'spotify:track:2E3quEE3yZnLsp1cbptT16'],
 ['Ray LaMontagne', 'Shelter', 'spotify:track:3V4GBwxo5UjAAjXbYES3x1'],
 ['Red Hot Chili Peppers',
  'Porcelain',
  'spotify:track:6WUjz18yIeTGDR1E69WHtf'],
 ['Rose Betts', 'Song to the Siren', 'spotify:track:43I0E943aSF6HMWPXcjLrw'],
 ['Rufus Wainwright', 'Hallelujah', 'spotify:track:2u6fRGcaBhpI4uNBHpGv0b'],
 ['RÜFÜS DU SOL', 'Innerbloom', 'spotify:track:6CGMZijOAZvTXG21T8t6R0'],
 ['Sergei Rachmaninoff',
  '14 Songs, Op. 34: No. 14, Vocalise',
  'spotify:track:6oc5alsRAftrM6W3THiulR'],
 ['Sigur Rós', 'Ára bátur', 'spotify:track:4O94lc0TZiFr9aU24zQ9LT'],
 ['Sigur Rós', 'Festival', 'spotify:track:3MiADOHOOwBCF9Pd5UCDXf'],
 ['Simon & Garfunkel',
  'The Sounds of Silence',
  'spotify:track:3CepTOU9Y7FezTt0CF3lCw'],
 ['Sleeping At Last', 'Mercury', 'spotify:track:3CZY55b05f682apPncuNHY'],
 ['Slowdive', 'When the Sun Hits', 'spotify:track:0oxYB9GoOIDrdzniNdKC44'],
 ['Spiritualized',
  'Ladies and gentlemen we are floating in space',
  'spotify:track:0fOjUafaAhJV16oRBgCtz7'],
 ['Steven Universe',
  'Here Comes a Thought (feat. Estelle & AJ Michalka)',
  'spotify:track:1SJhYNxOZSItqrB6upWyE6'],
 ['Stevie Wonder',
  'All In Love Is Fair',
  'spotify:track:7asmbWITwvuo1toSvoImnk'],
 ['Stratovarius', 'Black Diamond', 'spotify:track:1EoN7hqAsyuTUrBjJPzWJF'],
 ['Sufjan Stevens',
  'Visions of Gideon',
  'spotify:track:5c9qm0bMYawSyRNUTmUMs5'],
 ['Symphony X',
  'Candlelight Fantasia',
  'spotify:track:2JVrkN4Jfok1kjss6h6xOK'],
 ['Taylor Swift', 'august', 'spotify:track:3hUxzQpSfdDqwM3ZTFQY0K'],
 ['Taylor Swift',
  'the lakes - bonus track',
  'spotify:track:0eFQWVz0qIxDOvhLpZ40P7'],
 ['Taylor Swift', 'Labyrinth', 'spotify:track:0A1JLUlkZkp2EFrosoNQi0'],
 ['Taylor Swift', 'ivy', 'spotify:track:19CSr8rwW05VJL2F91KFNK'],
 ['Taylor Swift', 'my tears ricochet', 'spotify:track:1MgV7FIyNxIG7WzMRJV5HC'],
 ['Pyotr Ilyich Tchaikovsky',
  'Romeo and Juliet, TH 42 "Fantasy Overture"',
  'spotify:track:1wlLQUnS9EPVeu7Y0ABoK4'],
 ['The Album Leaf', 'Window', 'spotify:track:2aagSHemkQPt1MuBCkdXUc'],
 ['The Beatles',
  'Dear Prudence - Remastered 2009',
  'spotify:track:5NQYyej46WQkgCbnzGD21W'],
 ['The Beatles',
  'The Long One - Comprising of ‘You Never Give Me Your Money’, ’Sun King’/’Mean Mr Mustard’, ‘Her Majesty’, ‘Polythene Pam’/’She Came In Through The Bathroom Window’, ’Golden Slumbers’/ ’Carry That Weight’, ’The End’',
  'spotify:track:7e5QyScdgY4UFu3eYgy71w'],
 ['The Beatles',
  'And I Love Her - Remastered 2009',
  'spotify:track:65vdMBskhx3akkG9vQlSH1'],
 ['The Mixer Electronic Sound',
  'Close To You..',
  'spotify:track:0SzYkrpuA0kxcBaPx47zIs'],
 ['The Cinematic Orchestra',
  'That Home',
  'spotify:track:4jSuthawb7fitf3Wx3MqVM'],
 ['The Cinematic Orchestra',
  'To Build A Home',
  'spotify:track:3AqPL1n1wKc5DVFFnYuJhp'],
 ['The Dream Academy',
  'Please, Please, Please Let Me Get What I Want',
  'spotify:track:0smzrlYiBiecxfG8p5QDcQ'],
 ['The Irrepressibles',
  'In This Shirt',
  'spotify:track:14QX1ckbDIhqetvu5VZ3cP'],
 ['The Anix',
  'Cry Little Sister (The Lost Boys Theme)',
  'spotify:track:0QsaHoLRIUNc4kFXPQ7MLG'],
 ['The Magnetic Fields',
  'The Book of Love',
  'spotify:track:62ZYyXIsHVX5xVUj80arVu'],
 ['The National',
  'About Today - 2021 Remaster',
  'spotify:track:2w55zgyrG40D5ppKj4XxdK'],
 ['The National', 'I Need My Girl', 'spotify:track:50M7nY1oQuNHecs0ahWAtI'],
 ['Daniel & The River',
  'Into The Great Unknown',
  'spotify:track:3Qb70773uDv671Uf743JSx'],
 ['The Rolling Stones',
  'Gimme Shelter',
  'spotify:track:6H3kDe7CGoWYBabAeVWGiD'],
 ['The Smiths',
  "That Joke Isn't Funny Anymore",
  'spotify:track:48veRDsAEBOJJnVnnabV99'],
 ['The Cover Crew',
  'Bittersweet Symphony (Acoustic Version) [The Verve Cover]',
  'spotify:track:714doH50K9qbrE9py6izzV'],
 ['Thievery Corporation',
  'La Femme Parallel',
  'spotify:track:5fRm0HT0IiHyitxpxcrmyp'],
 ['Tina Turner', 'The Best', 'spotify:track:6pPWRBubXOBAHnjl5ZIujB'],
 ['U2', 'MLK - Live', 'spotify:track:3RxrdgLVyENtIesgO7M0yJ'],
 ['Ursine Vulpine', 'Wicked Game', 'spotify:track:3cdEXRnb7cZQSvKU7emY1h'],
 ['Van Morrison',
  'Moondance - 2013 Remaster',
  'spotify:track:683b4ikwa62JevCjwrmfg6'],
 ['Víg Mihály',
  'Öreg - From "Werckmeister Harmóniák"',
  'spotify:track:63wMgkXQuomlkW4an4O9b4'],
 ['Yeah Yeah Yeahs', 'Soft Shock', 'spotify:track:4Ku70eDzBKu5EP3oH7Ki9v'],
 ['Yiruma', 'May Be', 'spotify:track:42WpwYm7DYxbbVbUO35LUX'],
 ['Yiruma',
  'The Sunbeams,They Scatter',
  'spotify:track:0pjMeOto2TvTDJheqQxpaH'],
 ['Yo La Tengo',
  'My Little Corner of the World',
  'spotify:track:2bY66Hf5NbHJ8Ai8eNmJHG'],
 ['Unknown92', 'Untitled 1', 'spotify:track:2ZrnqByl5UzrWOGyOOlAmf'],
 ['Unknown Brain',
  'Perfect 10 - Unknown Brain & RudeLies VIP',
  'spotify:track:7xddfUj4h1dwZ7ooFNJGMM'],
 ['Ólafur Arnalds',
  'Happiness Does Not Wait',
  'spotify:track:0nYm42RuPWFaHWUXUdwHYm']]

In [102]:
addlist2 = [a[2] for a in addlist]

print (len(addlist2), 'items')

while(addlist2):
    sp.user_playlist_add_tracks(os.getenv('SPOTIFY_USERNAME'), 
                                playlist_id=playlist_id, 
                                tracks=addlist2[-100:])
    addlist2 = addlist2[:-100]
    print("added items, remaining ", len(addlist2))


Enter the URL you were redirected to: https://druce.ai/?code=AQDIg2MxBu8o2inFwEw3RJ8MLzYgxjgJPHM-ERNp2p4aPmNb6yDcLB-lNNoXHgVPsDXbs5iCBKcrns3wkljluRFb9nd0AlAuI8lhQSy6KyiG7gQhOlNctM99sZnJ_34nE5uUk-2re6ZO_NllLYRUS8J3_u1OuJUZCMxnYexyslC_ygN8tgeoWdjwo94
added items, remaining  103
added items, remaining  3
added items, remaining  0


In [103]:
len(addlist)

203